In [2]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session
import pandas as pd, requests, logging, os, base64
from datetime import datetime, timedelta

In [3]:
logger = logging.getLogger(__name__)
BASE_URL = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/'

In [9]:
def _encode_payload(page: int, trading_name: str, page_size: int = 100, language: str = "pt-br") -> str:
    """
    Codifica o payload para a requisição em Base64.
    """
    payload = f'{{"language":"{language}","pageNumber":{page},"pageSize":{page_size},"tradingName":"{trading_name}"}}'
    base64_bytes = base64.b64encode(payload.encode('ascii'))
    return str(base64_bytes, 'ascii')

def _fetch_data(trading_name: str):
    try:
        payload = _encode_payload(1, trading_name)
        url = BASE_URL + payload
        
        request = requests.get(url)
        if request.status_code != 200:
            raise Exception(f"Erro na requisição. Status code: {request.status_code}")
        response = request.json()
        if 'results' not in response:
            raise Exception(f"Erro na requisição. Mensagem: {response}")
        
        df = pd.DataFrame(response['results'])
        if 'page' not in response:
            logger.info("Apenas uma página de resultados")
            return df
        total_pages = response['page']['totalPages']
        for page in range(2, total_pages + 1):
            payload = _encode_payload(page, trading_name)
            url = BASE_URL + payload
            request = requests.get(url)
            if request.status_code != 200:
                raise Exception(f"Erro na requisição. Status code: {request.status_code}")
            response = request.json()
            df = pd.concat([df, pd.DataFrame(response['results'])], ignore_index=True)
        if df.empty:
            raise Exception("Nenhum resultado encontrado")
        return df
    except Exception as e:
        logger.error(f"Erro ao buscar dividendos: {str(e)}")
        raise

In [10]:
df = _fetch_data('Weg')

In [12]:
df['valueCash'] = df['valueCash'].str.replace('.', '').str.replace(',', '.').astype(float)
df['dateApproval'] = pd.to_datetime(df['dateApproval'], format="%d/%m/%Y")
df['ano'] = df['dateApproval'].dt.year
dt = df.groupby(['ano', 'typeStock']).agg({'valueCash': 'sum'}).reset_index()

In [14]:
dt

,ano,typeStock,valueCash
0,2004,ON,0.012941
1,2004,PN,0.012941
2,2005,ON,0.277941
3,2005,PN,0.277941
4,2006,ON,0.324412
5,2006,PN,0.324412
6,2007,ON,0.519824
7,2007,PN,0.179118
8,2008,ON,0.475529
9,2009,ON,0.420706
